In [1]:
import time
import threading
import multiprocessing
import multiprocess 
import itertools
import os
import logging
import random
import string
import requests
from functools import partial
from multiprocessing import Queue
from multiprocessing.pool import ThreadPool

# Definitions

In [3]:
# Processes are OS-level resource used to run and manage applications
# Processes have pid, list of open files and sockets, isolated memory,

In [4]:
# Special part of OS (scheduler) iterates over processes and give them time slot to run
TIME_SLOT = 0.5/(1000*1000)
while True:
    for process in processes:
        process.run_for(TIME_SLOT)

NameError: name 'processes' is not defined

In [5]:
# Process has 1 or more threads
# Threads have common memory

In [6]:
# 2 kinds of tasks:
- parallel execution: speedup
- background execution: 
    - daemon process: OS services
    - multithreaded process: notifications, monitor, watcher, etc

SyntaxError: invalid syntax. Perhaps you forgot a comma? (Temp/ipykernel_39056/3031161495.py, line 2)

In [7]:
Parallel execution can be done in 2 ways:
    - threads
    - processes
    
Which one is better?

SyntaxError: invalid syntax. Perhaps you forgot a comma? (Temp/ipykernel_39056/3321894035.py, line 1)

# Threads

In [2]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

TIME_TO_SLEEP = 4
        
def long_running_task(n=TIME_TO_SLEEP):
    print(threading.current_thread())
#     with timer('Elapsed 2: {}s'):
    time.sleep(n)

# what is it?
with timer('Executed in: {}s'):
#     long_running_task()
    long_running_task()


<_MainThread(MainThread, started 18672)>
Executed in: 4.009866952896118s


In [3]:
with timer('Executed in: {}s'):
    t1 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t2 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t1.start()
    t2.start()
    print(threading.current_thread())
    print('BEFORE T1 JOIN')
    t1.join() # be carefull
    print('BEFORE T2 JOIN')
    t2.join() 
    
print('next step')
# work with date

<Thread(Thread-8 (long_running_task), started 71068)>
<Thread(Thread-9 (long_running_task), started 69224)>
<_MainThread(MainThread, started 18800)>
BEFORE T1 JOIN
BEFORE T2 JOIN
Executed in: 2.011209487915039s
next step


In [4]:
def run_threads(func, data, workers):
    threads = [
        threading.Thread(target=func, args=(data / workers, ))
        for _ in range(workers)
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [5]:
workers = 10
DATA_SIZE = 10

with timer('Elapsed: {}s'):
    run_threads(long_running_task, DATA_SIZE, workers)

<Thread(Thread-10 (long_running_task), started 68976)><Thread(Thread-11 (long_running_task), started 35048)>

<Thread(Thread-12 (long_running_task), started 67124)>
<Thread(Thread-13 (long_running_task), started 61012)>
<Thread(Thread-14 (long_running_task), started 66620)>
<Thread(Thread-15 (long_running_task), started 69804)>
<Thread(Thread-16 (long_running_task), started 64836)>
<Thread(Thread-17 (long_running_task), started 35084)>
<Thread(Thread-18 (long_running_task), started 68968)>
<Thread(Thread-19 (long_running_task), started 63648)>
Elapsed: 1.0306055545806885s


In [12]:
def generate_task_queue(total_tasks, queue_size):
    task_queue = []
    remainder = total_tasks
    while remainder > 0:
        value = min(remainder, random.randint(1, total_tasks//queue_size))
        remainder -= value
        task_queue.append(value)
    return task_queue

assert sum(generate_task_queue(1000, 20)) == 1000
assert sum(generate_task_queue(1000, 1)) == 1000
assert sum(generate_task_queue(1000, 1000)) == 1000
assert sum(generate_task_queue(1000, 42)) == 1000

In [6]:
input_data = [DATA_SIZE / workers for _ in range(workers)]
input_data = [1, 2, 2, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
print(input_data)

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        pool.map(long_running_task, input_data)
#         for chunk in input_data:
#             long_running_task(chunk)

[1, 2, 2, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
<DummyProcess(Thread-20 (worker), started daemon 70196)><DummyProcess(Thread-21 (worker), started daemon 69256)>
<DummyProcess(Thread-22 (worker), started daemon 58852)>
<DummyProcess(Thread-23 (worker), started daemon 64828)>

<DummyProcess(Thread-24 (worker), started daemon 49100)>
<DummyProcess(Thread-25 (worker), started daemon 52460)>
<DummyProcess(Thread-26 (worker), started daemon 29912)>
<DummyProcess(Thread-27 (worker), started daemon 68636)>
<DummyProcess(Thread-28 (worker), started daemon 43624)>
<DummyProcess(Thread-29 (worker), started daemon 67152)>
Elapsed: 2.0162148475646973s


<img src="https://www.nginx.com/wp-content/uploads/2016/07/thread-pools-worker-process-event-cycle.png">

# Real world task

In [ ]:
def fetch_pic(num_pic):
# def fetch_pic(num_pic, path):
    url = 'https://picsum.photos/400/600'
    path = os.path.join(os.getcwd(), 'img')
    for _ in range(num_pic):
#         print(f"current image number: {i}")
        random_name = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'{path}/{random_name}.jpg', 'wb') as f:
                f.write(response.content)
                #print(f"Fetched pic [{os.getpid()}]: {f.name}")

In [ ]:
workers = 16
DATA_SIZE = 100

with timer('Elapsed: {}s'):
#     with multiprocessing.Pool(workers) as pool:
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        # input_data = [(DATA_SIZE // workers, './pics') for _ in range(workers)]
        pool.map(fetch_pic, input_data)
# 37 - 1 
# 6 - 8 

# IO vs CPU bound tasks

In [7]:
DATA_SIZE = 1_000_000
lst = []

def fill_data(n):
    # print(threading.current_thread())
    while n > 0:
        n -= 1
        lst.append(random.randint(1, 100))
        
with timer('Elapsed: {}s'):
#     fill_data(DATA_SIZE, lst)
    fill_data(DATA_SIZE)

Elapsed: 0.6632881164550781s


In [ ]:
1  2  M
   |  |
|     |
   |  |
|     |
   |
______


In [ ]:
with timer('Elapsed: {}s'):
#     t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
#     t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
    t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2,))
    t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

In [14]:
lst = []
workers = 8
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        result = pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

Elapsed: 0.004965305328369141s


NameError: name 'fill_data' is not defined

In [10]:
lst = []
workers = 8

DATA_SIZE = 1_000_000
lst = []

def fill_data(n):
    # print(threading.current_thread())
    while n > 0:
        n -= 1
        lst.append(random.randint(1, 100))
        
with timer('Elapsed: {}s'):
    with multiprocess.Pool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

Elapsed: 0.13699936866760254s


NameError: name 'lst' is not defined

In [ ]:
def factorize_naive(n):
    """ A naive factorization method. Take integer 'n', return list of
        factors.
    """
    if n < 2:
        return []
    factors = []
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1

    assert False, "unreachable"

In [ ]:
result = {}
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = [
            (n, factors)
            for n, factors in enumerate(pool.map(factorize_naive, input_data), 1)
        ]
    
print(len(result), result[:100])

In [ ]:
result = {}
workers = 16
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = [
            (n, factors)
            for n, factors in enumerate(pool.map(factorize_naive, input_data), 1)
        ]
    
print(len(result), result[:100])